## Workflow to generate ensemble parameter combinations

In [7]:
# this loads previoulsy stored environment
import dill
dill.load_session('opt_param_env.db')

In [8]:
# Load in necessary packages
import os
import pandas as pd
import numpy as np
import glob as glob
import datetime
from pyDOE import *

# personal functions
from funcs import unfold

### 1. Create dataframe with all parameters

In [9]:
# genetic parameter list
params = ['staygreen', 'rmax_lir', 'rmax_ltar', 'juv_leaves', # phenology
          'LAF', 'f', 'scatt', 'leaf_len', 'leaf_wid', # morphology
          'vcm_25', 'vpm_25', 'g0', 'g1', 't_opt', # physiology
          'psi_threshold'] # water stress

#### ensemble paramter range

In [10]:
param_range = {'staygreen':[1,6],
               'rmax_lir': [0.1, 0.5],
               'rmax_ltar': [0.1, 1.5],
               'juv_leaves': [10, 15],
               'LAF': [1, 3],
               'f': [0, 1],
               'scatt': [0, 1],
               'leaf_len': [100, 200],
               'leaf_wid': [10, 50],
               'vcm_25': [40, 60],
               'vpm_25': [60, 80],
               'g0': [0.02, 0.1], # can check the range of these values from literature maybe?
               'g1': [2, 4],      # ball-berry slope & intercept
               't_opt': [20, 28],
               'psi_threshold': [-5, -1]}

### 2. Use Latin hypercube to find parameter combinations

info on this method here: https://pythonhosted.org/pyDOE/randomized.html

In [33]:
sample = lhs(15, samples=100)
df_params = pd.DataFrame(sample, columns=params)

for i in params: 
    df_params[i] = unfold(df_params[i], param_range[i][0], param_range[i][1])

# constraining leaf number to be integers only
df_params.juv_leaves = [round(x) for x in df_params.juv_leaves]
df_params.head()

,staygreen,rmax_lir,rmax_ltar,juv_leaves,LAF,f,scatt,leaf_len,leaf_wid,vcm_25,vpm_25,g0,g1,t_opt,psi_threshold
0,3.580975,0.198711,0.351016,13,2.876520,0.064568,0.106130,106.190190,13.885371,48.347382,70.620698,0.043747,3.556952,26.675907,-3.198173
1,5.990791,0.261375,0.367098,12,1.324976,0.483678,0.597002,193.567761,48.201314,41.062333,65.901951,0.021885,2.876227,21.290204,-1.815574
2,4.915283,0.112661,1.346119,11,2.588981,0.862133,0.614563,143.936102,42.389777,55.133019,75.452608,0.020834,2.551809,21.559385,-3.426829
3,1.988260,0.248188,0.639833,11,2.560768,0.222129,0.754981,161.596353,27.072121,47.971242,67.242830,0.030859,3.960681,21.965958,-4.896455
4,2.769490,0.436952,0.500761,14,1.137164,0.805315,0.602996,159.439111,29.189599,46.337389,70.816556,0.064980,2.733855,25.771664,-1.057574


### 3. Setting up planting dates

In [12]:
# planting date range
season_start = datetime.datetime(1990,3,1)
season_end = datetime.datetime(1990,5,31)
dates = np.arange(season_start, season_end, datetime.timedelta(3))

pdates = []
for i in dates:
    pdates.append((str(i)[4:10]))

In [13]:
dill.dump_session('opt_param_env.db')